# New Raster Mask

Created 2019-01-15
This program takes the code from Raster Mask and cleans it up for use to do two things:

1. Create new rasters with a simple mask applied
2. Complete connected components of two masks

Check crs of raster
'EPSG:4326' ... or EPSG: 54009

In [1]:
import rasterio
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt

### Inputs

In [2]:
temp_data = '../../../temp_data/'
raster_interim = '../../../data/interim/'
raster_raw = '../../../data/'
downloads = '/Users/cascade/Downloads/'

### Functions

In [3]:
# Let's try to make the mask directy with rasterio

def raster_mask_poly(raster, band, density, crs):
    """Fucntion makes numpy array for of raster then applies mask.
    requires rasterio object, band number, and density as float on int and crs of raster
    returns dict of polygons and numpy mask
    """

    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros 
    mask[mask >= density] = 1 
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326) removed 2019-01-15
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return mask, geom, poly_gdf

In [4]:
def raster_mask(raster, band, density):
    "Function returns raster mask"
    
    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros and ones
    mask[mask >= density] = 1
    
    return mask

In [5]:
# Let's try to make polygons

def raster_poly(raster, band, crs):
    """Fucntion makes ploygons for of raster returns dict of polygons"""

    mask = raster.read(band)
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return poly_gdf

In [6]:
def raster_gpd(array, raster, crs):
    """ 
    function takes an np array, raster
    and returns polygons from the np array in the raster CRS
    """
    for geom, val in rasterio.features.shapes(array, transform = raster.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326). Removed # WGS84 (EPSG 4326) 
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) 

    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)
    
    return poly_gdf

In [7]:
def raster_write(meta, array, file_out):
    """ function to write out a raster file with an np array
    requires meta data for raster, np array & file out path and name
    """
    
    kwargs = meta

    # Update kwargs (change in data type)
    kwargs.update(dtype=rasterio.float32, count = 1)

    with rasterio.open(file_out, 'w', **kwargs) as dst:
        dst.write_band(1, array.astype(rasterio.float32))

### Connected Pixels 1500 - 300

In [12]:
import os

os.getcwd()
os.chdir('/Users/cascade/Github/Pop-ERL//')
os.getcwd()

from src import raster_funcs

In [13]:
file = 'LS15_w001001_Clip'
raster = rasterio.open('data/interim/'+file+'.tif')
crs = raster.crs

In [14]:
mask300 = raster_mask(raster, 1, 300)
mask1500 = raster_mask(raster, 1, 1500)

In [15]:
mask1500c300 = raster_funcs.remove_isolated_pixels(mask1500, mask300)

In [16]:
poly_gdf_1500c300 = raster_gpd(mask1500c300, raster, crs)

In [ ]:
print((poly_gdf_1500c300.head(6)))
print(len(poly_gdf_1500c300))

In [ ]:
# drop first row, which is bounding box of Africa
poly_gdf_1500c300_drop = poly_gdf_1500c300.drop(poly_gdf_1500c300.index[0], axis = 0)
print((poly_gdf_1500c300_drop.head(6)))
print(len(poly_gdf_1500c300_drop))

In [ ]:
poly_gdf_1500c300_drop.to_file(downloads+file+'_1500c300.shp', driver='ESRI Shapefile')

### 300 ppl km2

In [ ]:
mask300m, geom300, poly_gdf300 = raster_mask_poly(raster, 1, 300, crs)

In [ ]:
print(poly_gdf300.head(6))
print(len(poly_gdf300))

In [ ]:
# drop first row, which is bounding box of Africa and the write 
poly_gdf300 = poly_gdf300.drop(poly_gdf300.index[0], axis = 0)
print(poly_gdf300.head(6))
print(len(poly_gdf300))

In [ ]:
#write out polygons 
# poly_gdf300.to_file(temp_data+'AFR_PPP_2015_adj_v2_300.shp', driver='ESRI Shapefile')